In [1]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from opentelemetry import trace
from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from config_helper import ConfigHelper
from pipeline import DocumentPipeline
from localrag import LocalRAG

# We are using Aspire. Of course we will see the telemetry and logs in our dashboard!
# see config_helper.py for the not to tidy details.
tracer = trace.get_tracer(__name__)

config_helper = ConfigHelper(False)


In [2]:
# Ingest a GitHub repository and import into our vector store

# Initialize the pipeline
pipeline = DocumentPipeline(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config
)

###Process git repository
repository="https://github.com/dotnet/docs-aspire"
with tracer.start_as_current_span(f"Starting ingesting {repository}"):
    pipeline.process_repository(repository)

# Or process local directory
#pipeline.process_local_directory("./docs")


/home/jovyan/work/code/pipeline.py:37: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


Total parts found: 439
Total files processed: 219
Sample file paths:
 1. /README.md
 2. /docs/diagnostics/overview.md
 3. /docs/breadcrumb/toc.yml
Added 2 chunks for file /docs/breadcrumb/toc.yml
Added 73 chunks for file /docs/toc.yml
Added 5 chunks for file /docs/compatibility/toc.yml
Added 7 chunks for file /docs/zones/zone-pivot-groups.yml
Added 50 chunks for file /docs/reference/aspire-faq.yml
Added 72 chunks for file /docs/index.yml
Added 6 chunks for file /docs/whats-new/toc.yml
Added 18 chunks for file /docs/whats-new/index.yml
Added 22 chunks for file /.repoman.yml
Added 1 chunks for file /.pre-commit-config.yaml
Added 186 chunks for file /.github/dependabot.yml
Added 11 chunks for file /.github/workflows/quest-bulk.yml
Added 5 chunks for file /.github/workflows/markdownlint.yml
Added 13 chunks for file /.github/workflows/snippets5000.yml
Added 8 chunks for file /.github/workflows/clean-repo.yml
Added 16 chunks for file /.github/workflows/scorecards.yml
Added 10 chunks for file

In [ ]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "Why should I know about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            print("\nRelevant chunks:")
            #with tracer.start_as_current_span("rag get chunks"):
            # chunks = rag.get_relevant_chunks(question, k=5)
            # for i, chunk in enumerate(chunks, 1):
            #     print(f"\nChunk {i}:")
            #     print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
            #      print(f"Content: {chunk.page_content[:200]}...")
                
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=6)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config
)
with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)

Question: Why should I know about .Net Aspire?

Relevant chunks:

Generated Answer:


/home/jovyan/work/code/localrag.py:42: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


.NET Aspire is a set of libraries and tools designed to make it easier to build observable, production-ready applications, especially for cloud-native apps. 

Here's why you should know about .NET Aspire:

* **Building Observability:** It provides features that help you create more robust and understanding applications by introducing observability mechanisms.
* **Simplifies Distributed App Development:**  It offers tools to manage complex distributed applications, making it easier to develop and run them efficiently. 
* **Cloud-Native Focus:** .NET Aspire is built with cloud-native development in mind, focusing on microservices and efficient service integration.

Essentially, knowing about .NET Aspire will help you:

* **Improve your understanding of building modern cloud-native applications.**
* **Utilize its tools to build more robust and observable apps.** 
* **Develop custom resources for .NET Aspire projects with greater technical knowledge.**


Let me know if you have any other q